In [4]:
!pip uninstall torchaudio -y

Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121


In [1]:
import torch

print(torch.__version__)

2.5.1+cu121


In [4]:
!pip install vllm==0.6.4.post1
!pip install triton=3.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.1 MB/s eta 0:00:00


In [5]:
import time
import torch
import pandas as pd
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
import vllm

In [6]:
MAX_LENGTH = 1000
BATCH_SIZE = 2
MODEL_NAME = "llm-jp/llm-jp-3-3.7b-instruct"
IS_DEBUG = False

In [8]:
llm = vllm.LLM(
    MODEL_NAME, # "deepseek-ai/deepseek-math-7b-instruct"
    tensor_parallel_size=1, # 2, 4
    gpu_memory_utilization=0.95,
    trust_remote_code=True,
    dtype="half",
    #enforce_eager=True,
    #max_model_len=1024,
    #enable_lora=True,
    #quantization="AWQ",
    #quantization="bitsandbytes",
    #load_format="bitsandbytes",
)
tokenizer = llm.get_tokenizer()

WARNING 11-20 14:47:54 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 11-20 14:47:54 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 11-20 14:47:54 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='llm-jp/llm-jp-3-3.7b-instruct', speculative_config=None, tokenizer='llm-jp/llm-jp-3-3.7b-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, col

model-00002-of-00002.safetensors:   0%|          | 0.00/2.57G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 11-20 14:49:59 model_runner.py:1077] Loading model weights took 7.0482 GB
INFO 11-20 14:50:01 worker.py:232] Memory profiling results: total_gpu_memory=14.75GiB initial_memory_usage=7.17GiB peak_torch_memory=7.98GiB memory_usage_post_profile=7.20GiB non_torch_memory=0.14GiB kv_cache_size=5.89GiB gpu_memory_utilization=0.95
INFO 11-20 14:50:01 gpu_executor.py:113] # GPU blocks: 1147, # CPU blocks: 780
INFO 11-20 14:50:01 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 4.48x
INFO 11-20 14:50:05 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-20 14:50:05 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-2

In [9]:
tokenizer = llm.get_tokenizer()

from datasets import load_dataset
tasks = load_dataset("elyza/ELYZA-tasks-100", split="test")

messages_list = [
    [{"role": "user", "content": tasks["input"][i]}] for i in range(len(tasks))
]
prompts = [line[0]["content"] for line in messages_list]
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True) for messages in messages_list]
sampling_params = vllm.SamplingParams(
    temperature=0.5,
    max_tokens=1024,
)
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)
for prompt, response in zip(prompts, outputs):
    print("prompt:", prompt)
    print("output:", response.outputs[0].text.strip())
    print("-"*80)

import json
data = [{
    "task_id": f"task_{i+1}",
    "input": prompts[i], # あってもなくてもよい
    "output": outputs[i].outputs[0].text.strip()
} for i in range(len(tasks))]
file_path = 'output.jsonl'
with open(file_path, 'w', encoding='utf-8') as file:
    for entry in data:
        json.dump(entry, file, ensure_ascii=False)
        file.write('\n')

README.md:   0%|          | 0.00/8.37k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/115k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Processed prompts: 100%|██████████| 100/100 [00:32<00:00,  3.04it/s, est. speed input: 253.68 toks/s, output: 420.04 toks/s]

prompt: 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
output: もちろんです。以下に、仕事の熱意を取り戻すためのアイデアを5つ挙げます。

1. 目標設定と進捗確認:
   - 定期的に目標を見直し、達成度を確認することで、モチベーションを維持できます。
   - 小さな成功体験を積み重ねることで、自信を取り戻すことができます。

2. 環境の整備:
   - 作業環境を整え、快適な状態を保つことで、集中力が向上します。
   - 必要なツールや資料を手元に揃えておくことも重要です。

3. 休憩とリフレッシュ:
   - 定期的に短い休憩を取り入れることで、リフレッシュし、再びエネルギーを取り戻すことができます。
   - 軽い運動や散歩も効果的です。

4. コミュニケーション:
   - 上司や同僚とオープンにコミュニケーションを取ることで、問題解決や新しいアイデアの共有が可能になります。
   - フィードバックを求めることで、自分の仕事の方向性を見直すことができます。

5. 自己啓発とスキルアップ:
   - 新しいスキルや知識を習得することで、仕事に対する興味や情熱を取り戻すことができます。
   - オンラインコースやセミナーに参加するのも良い方法です。

これらのアイデアを実践することで、仕事の熱意を取り戻すことができるでしょう。
--------------------------------------------------------------------------------
prompt: クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
output: ある晴れた日の午後、クマのロッコは深い森の奥から海辺へと足を踏み入れました。初めて見る景色に心を躍らせながら、彼は砂浜にたどり着きました。そこには一匹のアザラシ、名前をライナスが泳いでいました。ライナスはふわふわの毛皮を持ち、大きな耳とつぶらな瞳が愛らしい生き物でした。

ロッコはライナスに近づき、優しく声をかけました。「こんにちは、ライナス。君はここで何をしているの？」

ライナスは少し驚いた表情を見せましたが、すぐに笑顔になりました。「こんにちは、ロッコ。僕はただ泳ぎ疲れて休んでいるんだ。でも、君が来